In [ ]:
import rosbag
from cv_bridge import CvBridge, CvBridgeError
import cv2

from matplotlib import pyplot as plt
from pathlib2 import Path
import pandas as pd
import numpy as np


DEPTH_TOPIC = '/camera/aligned_depth_to_color/image_raw'
COLOR_TOPIC = '/camera/color/image_raw'
JOYSTIC_TOPIC = '/joy'
BAGS_PATH = Path('fuw_20190130')

IMAGE_WIDTH, IMAGE_HEIGHT = 480, 640

In [ ]:
bag_file = 'test_2019-01-30-14-27-02.bag'



bag_filepath = BAGS_PATH / bag_file


bag = rosbag.Bag(str(bag_filepath))

topics_info = bag.get_type_and_topic_info().topics

depth_msg_count = topics_info[DEPTH_TOPIC].message_count
color_msg_count = topics_info[COLOR_TOPIC].message_count
joy_msg_count = topics_info[JOYSTIC_TOPIC].message_count

# In what follows, I'm assuming that the number of messages from
# the joypad is greater than for color image, and that color image
# has more messages than depth image
assert (
    joy_msg_count >= color_msg_count >= depth_msg_count
)
    
joy_index = 0
depth_index = 0

depth_storage = np.zeros(
    (depth_msg_count, IMAGE_WIDTH, IMAGE_HEIGHT),
    dtype='uint16'
)
angle_storage = np.zeros(depth_msg_count)
throttle_storage = np.zeros(depth_msg_count)

events = []

bridge = CvBridge()
for topic, msg, t in bag.read_messages():
    events.append({'time': t.to_nsec(), topic: 1})
    if topic == DEPTH_TOPIC:
        depth_storage[depth_index] = bridge.imgmsg_to_cv2(
            msg,
            desired_encoding='passthrough'
        )
        depth_index += 1
    if topic == JOYSTIC_TOPIC and depth_index > joy_index:
        axes = msg.axes
        angle = axes[0]
        throt = axes[3]
        # Typically, the transformation from joy messages
        # ro steer and throttle is as follow:
        # angle = 1500 - int(505*angle)
        # throt = 1500 + int(120*throt)
        # but I'm logging the values "as-is" (as was
        # the PWM signal sent by the microcontroller to
        # the servo and the ESC)
        angle_storage[joy_index] = angle
        throttle_storage[joy_index] = throt
        joy_index += 1
    if depth_index > joy_index + 1:
        # There are no messages when the car is going straight and
        # throttle is 0 -- we need to encode that ourselves
        angle_storage[joy_index] = 0
        throttle_storage[joy_index] = 0
        joy_index += 1

bag.close()
        
DF_events = pd.DataFrame(events).fillna(0)
DF_events.plot.line(x='time', subplots=True);

time_diffs = np.diff(DF_events['time']) / float(10**9)
print('Longest time_diff: {} [s]'.format(time_diffs.max()))
print('Mean time_diff: {} [s]'.format(time_diffs.mean()))


output_suffix = '{directory}_{bag_file}'.format(
    directory=BAGS_PATH,
    bag_file=bag_file.replace('.bag', '')
)


# Save logs
DF_logs = pd.DataFrame({
    'steer': angle_storage,
    'throttle': throttle_storage,
})

assert len(DF_logs) == depth_msg_count

log_filename = 'extracted_data/logs_{}.csv'.format(output_suffix)
DF_logs.to_csv(log_filename, index=False)


# Save images
depth_filename = 'extracted_data/depth_{}.npy'.format(output_suffix)
np.save(depth_filename, depth_storage)

## Images may look like they're becoming darker, but it's actually a consequence of [colormap normalization](https://matplotlib.org/users/colormapnorms.html)

In [ ]:
import matplotlib as mpl
import matplotlib.gridspec as gridspec


def plot_image(image, normalizing_const=20, num_bins=25, max_dist=16):
    mm_in_m = 1000.
    
    fig = plt.figure(constrained_layout=True)
    gs = gridspec.GridSpec(2, 2, figure=fig)

    ax = fig.add_subplot(gs[0, 0])
    ax.imshow(image, cmap='gray')
    ax.set_title('Imshow with colornorm')
    
    ax = fig.add_subplot(gs[0, 1])
    normed_image = (image / float(normalizing_const)).astype(int)
    ax.imshow(normed_image, cmap='gray', norm=mpl.colors.NoNorm())
    ax.set_title('Imshow without colornorm')
    
    ax = fig.add_subplot(gs[1, :])
    ax.hist(image.flatten() / mm_in_m, bins=num_bins, range=(0, max_dist));
    ax.set_title('Histogram of distances in the depth map')
    ax.set_xlabel('Distance [m]')
    
    plt.show()

In [ ]:
plot_image(depth_storage[19])

In [ ]:
plot_image(depth_storage[20])